# Long Short-Term Memory

Entra uma sequência de inputs $\{x_1, x_2, ...\}$

As saídas são $\{h_1, h_2, ...\}$

Para gerar $h_t$, usa-se $x_t$ e $h_{t-1}$, com $h_0 = 0$ // até aqui é **Recurrent** Neural Network ou RNN

<div align="center">

![RNN comum](RNNcomum.jpg)

</div>

Além disso, o sistema possuirá memória $c_t$, e para controlarmos a memória faremos "três redes neurais extras", de forma que o LSTM, no final, são 4 em 1.

Quais três? 

- Uma que controle **quais** informações salvar na memória: $i_t$
- Uma que decide se/como usar a memória: $o_t$
- Por fim, uma que decide se apaga ou não a memória: $f_t$

Todas irão agir apenas sob a forma de "portas lógicas", sendo vetores em $(0,1)^n $ (para algum $n$) e fazendo produto elemento a elemento com outros vetores. Por isto foram denotados por $-_t$, que representa tal porta lógica no instante $t$.

Denotaremos o resultado da rede neural "original" (que não é nenhuma das três acima) por $\overline{c}_t$. Isto se deve ao fato de $\overline{c}_t$ sofrer alterações das outras três **após** o resultado desta rede neural, até que enfim obtemos $h_t$.

Especificamente:
$$ f_t = \sigma_g(W_fx_t + U_fh_{t-1} + b_f), \\ i_t = \sigma_g(W_ix_t + U_ih_{t-1} + b_i),
 \\ o_t = \sigma_g(W_ox_t + U_oh_{t-1} + b_o), \\ \overline{c}_t = \sigma_c(W_cx_t + U_ch_{t-1} + b_c),
 \\ c_t = f_t \odot c_{t-1} + i_t \odot \overline{c}_t, \\ h_t = o_t \odot \sigma_h(c_t),$$

onde $\sigma_g = \frac{1}{1+e^{-x}}$ é a sigmoid (portanto entre 0 e 1), $\sigma_c = \frac{e^x - e^{-x}}{e^x + e^{-x}}$ é a tangente hiperbólica, e $\sigma_h$ pode ser a tangente hiperbólica ou identidade. 

Existem algumas variações de LSTM, uma delas é "peephole", onde o output $h_t$ não é usado como em RNN's clássicas, normalmente substituído pela memória $c_t$.



## Imports

In [47]:
import numpy as np

%load_ext line_profiler

def relu(x):
    return (x > 0)*x

def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_2(x):
    y = np.exp(x)
    return y/(y+1)

def entropy(x, y):
    return - y * np.log(x) - (1 - y) * np.log(1 - x)

def quaderror(x, y, f):
    return np.dot(f(x) - y, f(x) - y)/2

# %lprun -f tanh tanh(np.arange(0,500))

# %lprun -f tanh_2 tanh_2(np.arange(0,500))

# %lprun -f tanh_3 tanh_3(np.arange(0,500))

%timeit relu(np.arange(-100,100))


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
4.5 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [4]:
yhat = 10